# 1. Úkoly k týdnu 08

Úkoly navazují na notebooky `08a` až `08g`.

## 1.1 SciPy Special

In [ ]:
import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt

1. Výpočet Eulerovy konstanty

- Spočtěte Eulerovu konstantu $\gamma$ pomocí funkce `digamma` z modulu `scipy.special`.
    - Platí $\gamma = -\psi(1)$, kde $\psi$ je digamma funkce.
- Porovnejte výsledek s limitní definicí pro $n = 100, 1000, \dots$:

$$\gamma = \lim_{n \to \infty} \left(-\ln(n) + \sum_{k=1}^{n} \frac{1}{k}\right)$$

2. Vykreslete Beta funkci na intervalu $[1, 2] \times [1, 2]$.

## 1.2 SciPy Sparse

In [ ]:
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import numpy as np
import matplotlib.pyplot as plt

1. Sestavení řídké matice: vytvořte sparse matici `A` ve formátu CSR, která reprezentuje
```
[[1 1 0 0]
 [1 3 1 0]
 [0 1 5 1]
 [0 0 1 7]]
```
- Použijte sestavení pomocí seznamu indexů a hodnot.

2. Řešení série soustav lineárních rovnic se stejnou maticí `A` a různými pravými stranami `b`:
- sestavte LU faktorizaci matice `A`
- vyřešte soustavu pro pravé strany `b1 = [1, 2, 3, 4]` a `b2 = [4, 3, 2, 1]` pomocí spočtené LU faktorizace

3. Spektrální rozklad:
- spočítejte největší vlastní číslo a příslušný vlastní vektor matice `A` z úlohy 1

## 1.3 SciPy FFTpack

In [ ]:
from IPython.display import Audio
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack

fs, signal = sp.io.wavfile.read('butterfly_noise.wav')
signal = signal / np.max(np.abs(signal))
Audio(data=signal, rate=fs)

1. Pokuste se odstranit šum z nahrávky pomocí Fourierovy transformace:
- spočtěte Fourierovu transformaci nahrávky (`fft`)
- vykreslete amplitudové spektrum (`abs`) a zvolte hranici mezi šumem a signálem
- odstraňte šum ve spektru a spočtěte inverzní Fourierovu transformaci (`ifft`)
- přehrajte reálnou složku výsledku: `Audio(data=np.real(filtered_signal), rate=fs)`

## 1.4 SciPy Optimize

In [ ]:
import scipy.optimize as opt

1. Nalezněte globální maximum funkce $$f(x) = \exp(-x^2) - 1.1 \cdot \exp(-(x-2)^2)$$

In [ ]:
x = np.linspace(-5, 5, 1000)
y = np.exp(-x**2) - 1.1 * np.exp(-(x - 2)**2)
plt.plot(x, y)
plt.grid()

2. Nalezněte všechny kořeny funkce $f(x) = 7.875 \cdot x^5 - 8.75 \cdot x^3 + 1.875 \cdot x$ na intervalu $[-1, 1]$.

In [ ]:
def f(x): return 7.875 * x**5 - 8.75 * x**3 + 1.875 * x


## 1.5 SciPy Interpolate

1. Spočtěte lineární a kubickou interpolaci bodů $(0, 0), (1, 1), (2, 0), (3, 1)$ na intervalu $[0, 3]$ a vše vykreslete.

2. Pro stejné body spočtěte aproximaci polynomem stupně 2 a vykreslete.

## 1.6 SciPy Integrate

In [ ]:
import scipy.integrate as spi

1. Spočtěte integrál funkce $f(x) = x^2$ na intervalu $[1, 2]$ pomocí funkce `quad`.

2. Spočtěte integrál funkce $f(x) = \exp(-x)$ na intervalu $[0, \infty)$ pomocí funkce `quad`.

3. Spočtěte průběh radioaktivního rozpadu:
- rychlost úbytku radioaktivní látky (hmotnosti) je úměrná aktuální hmotnosti
$$m'(t) = -\lambda m(t)$$
    - kde $\lambda$ je konstanta odvozená z poločasu rozpadu
- spočtěte hmotnost látky v čase $t \in [0, 100]$ pro $m(0) = 1$ a $\lambda = 0.1$ pomocí funkce `odeint`